# Archive my tweets

- badges: true
- categories: [code, twitter]


I want to extract all the tweets I've ever written and convert them to small markdown files so they show up as "posts" on this website.

Posts are organized by date, in the traditional blogging format. But so are tweets, kind of? They're listed chronologically anyway. Maybe I could make one post for each day, and then have all the tweets listed on that page. Each one could have its tweet ID as a header, thus having an internal link. Tweets can link to the tweets that precede them, and maybe even backlink to tweets that follow.

It's not quite block references, but as a way of keeping my second brain under my ownership it should work. And this way if anyone wants to cancel me they'll have a convenient search box and permalinks for it. Even if my account gets deleted, my bad takes can stay up.

In [1]:
#hide_output
import twint
import nest_asyncio
nest_asyncio.apply()


c = twint.Config()

c.Username = 'deepfates'
tweets = []
c.Store_object = True
c.Store_object_tweets_list = tweets


twint.run.Search(c)

1429133754598109189 2021-08-21 11:30:13 -0600 <deepfates> @Knipps @longnow yess Of course it was in one of these talks, have listened to so many of them that they blur together.  I love this part: Stewart Brand: “Are there rituals that muddlers…?" Brian Eno: “Muddlers like me surrender to the muddle, and love the process of surrendering to the muddle.”
1429133302041034762 2021-08-21 11:28:25 -0600 <deepfates> @aeranothaur This is cool because it creates a meta language for designing poetic programming languages? is that right?  It does look like free verse though. I want something more like rap
1429132776880623616 2021-08-21 11:26:20 -0600 <deepfates> @Knipps where did you find this. I can't locate the original quote any freaking where
1429132680113926149 2021-08-21 11:25:57 -0600 <deepfates> @s_sempervirens multi-threaded
1429132633297010699 2021-08-21 11:25:46 -0600 <deepfates> @VividVoid_ whoo 😮‍💨 I went to so many meetings in Portland about buying land for a 21st century village. c

In [2]:
#better save that list to disk before i mess around too much more
import pickle

with open('../assets/tweets.pkl', 'wb') as f:
    pickle.dump(tweets, f)

In [3]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

In [4]:
len(self_tweets)

368

In [5]:
t = tweets[5]

In [6]:
t.conversation_id, t.datestamp, t.datetime, t.id, t.likes_count, t.link, t.mentions, t.photos, t.quote_url, t.replies_count, t.reply_to, t.retweet, t.retweet_date, t.retweet_id, t.retweets_count, t.source, t.thumbnail, t.timestamp, t.timezone, t.tweet, t.urls, t.user_id, t.user_id_str, t.user_rt, t.user_rt_id, t.username, t.video

('1429127838419099649',
 '2021-08-21',
 '2021-08-21 11:22:58 MDT',
 1429131930042261508,
 0,
 'https://twitter.com/deepfates/status/1429131930042261508',
 [],
 [],
 '',
 0,
 [{'screen_name': 'max1231235564',
   'name': 'Firstname Bunchofnumbers',
   'id': '1409998134885847041'}],
 False,
 '',
 '',
 0,
 '',
 '',
 '11:22:58',
 '-0600',
 '@max1231235564 I think a lot of people have realize that disasters happen and we should prep for it in the last couple years. hopefully they have too  &amp;often people need to feel that they have come to a conclusion independently. remember to give them space to think   💪 deep breaths, u got this',
 [],
 3315205122,
 '3315205122',
 '',
 '',
 'deepfates',
 0)

In [7]:
import requests
import shutil

In [8]:
def dl_image(url):
    filename = '../images/' + url.split('/')[-1]
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        return(filename)
    else:
        return(None)
    
def image_template(filename):
    return(f'![image from twitter](/{filename[2:]})\n')

    
def get_tweet(t):
    if t.photos == []:
        img_md = ''
    else:
        img_list = [dl_image(url) for url in t.photos]
        img_md = '\n'.join([image_template(o) for o in img_list])

    return(f'''
#### <a href = "{t.link}">*{t.timestamp}*</a>

<font size="5">{t.tweet}</font>

{img_md}

🗨️ {t.replies_count} ♺ {t.retweets_count} 🤍  {t.likes_count}   

---
    ''')

def get_md(tweets, date):
    tweets_text = ''.join(t for t in tweets)
    return(f'''---
title: Tweets for {date}
layout: post
toc: true
comments: false
search_exclude: false
categories: [tweets]
---

{tweets_text}
            ''')

In [9]:
from IPython.display import Markdown

In [10]:
Markdown(get_tweet(t))


#### <a href = "https://twitter.com/deepfates/status/1429131930042261508">*11:22:58*</a>

<font size="5">@max1231235564 I think a lot of people have realize that disasters happen and we should prep for it in the last couple years. hopefully they have too  &amp;often people need to feel that they have come to a conclusion independently. remember to give them space to think   💪 deep breaths, u got this</font>



🗨️ 0 ♺ 0 🤍  0   

---
    

### 1426180149029085188
<a href = "https://twitter.com/deepfates/status/1426180149029085188">*07:53:39*</a>

yeah babe, I'm almost ready to go. Just let me get my boots on  https://t.co/QByQNB0H0W

![image from twitter](../images/E8rPSWwX0AYVBW5.jpg)

🗨️5 ♻️4  ❤️55    


In [11]:
yesterday = '2021-08-12'
y_tweets = [tw for tw in tweets if tw.datestamp == yesterday]
len(y_tweets)

12

In [12]:
y_sorted = sorted(y_tweets, key=lambda x: x.datetime)
# [tweet.tweet for tweet in y_sorted]

Too many replies! Let's limit to just mine for now

In [13]:
y_md = get_md([get_tweet(t) for t in y_sorted if "@" not in t.tweet], yesterday)

In [14]:
# y_md

In [15]:
with open(f'../_posts/tweets/{yesterday}-tweets.md', 'w') as f:
    print(y_md, file=f)

Okay, that'll do for now. It prints a chronological page of tweets for each day. Linking, video and oter people's tweets will have to come later.

I'll wrap that behavior in a function and pass it my tweets and a set of dates when i have tweeted.

In [16]:
def write_day_page(day, tweets):
    tweets = [tw for tw in tweets if tw.datestamp == day]
    sorted_tweets = sorted(tweets, key=lambda x: x.datetime)
    md = get_md([get_tweet(t) for t in sorted_tweets], day)
    with open(f'../_posts/tweets/{day}-tweets.md', 'w') as f:
        print(md, file=f)

In [17]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

In [18]:
len(self_tweets)

368

In [19]:
len([tw for tw in self_tweets if tw.datestamp == yesterday])

2

In [20]:
write_day_page(yesterday, self_tweets)

In [21]:
days = set([t.datestamp for t in self_tweets])

In [22]:
from tqdm import tqdm

In [23]:
for day in tqdm(days):
    write_day_page(day, self_tweets)

100%|██████████| 10/10 [00:27<00:00,  2.71s/it]


I would also liek to do analysis to see how often I tweet. And make a big list of links. Maybe next time.

For now you can find these secret tweet archives by searching in the [Explore](/explore) page